## Data

In [13]:
with open("input.txt", "r") as infile: 
    contents = infile.read().strip().split("\n")

contents[0:4]

['132, 308', '325, 300', '310, 231', '177, 248']

## Part 1

In [57]:
coordinates = list(set([(int(x), int(y)) for (x, y) in [line.split(',') for line in contents]]))
coordinates[0:4]

[(162, 176), (250, 117), (354, 92), (235, 190)]

In [58]:
len(coordinates)

50

In [59]:
def distance(A, B):
    """returns the Manhattan distance between two points A and B"""
    return abs(A[0]-B[0]) + abs(A[1]-B[1])

In [60]:
xmin = min([x for (x, y) in coordinates])
xmax = max([x for (x, y) in coordinates])
ymin = min([y for (x, y) in coordinates])
ymax = max([y for (x, y) in coordinates])

xmin, xmax, ymin, ymax

(41, 354, 42, 353)

In [71]:
#at each location in the matrix are the location which are equally distant
matrix = dict()

for x in range(xmin, xmax+1): 
    for y in range(ymin, ymax+1): 
        matrix[x,y] = [coordinates[0]]

for i, location in enumerate(coordinates[1:]): 
    for x in range(xmin, xmax+1): 
        for y in range(ymin, ymax+1):
            this = distance(location, (x, y))
            that = distance(matrix[x, y][0], (x, y))
            
            if this < that:
                matrix[x, y] = [location]
            elif this == that: 
                matrix[x, y].append(location)
                
matrix[300,300]

[(325, 300)]

In [62]:
#sanity check 
#find a location for which several locations are equi-distant
for x in range(xmin, xmax+1): 
    for y in range(ymin, ymax+1):
        if len(matrix[x, y]) > 2: 
            print((x, y), matrix[x, y])
            break
    else: 
        continue
    break

(62, 279) [(85, 273), (56, 256), (70, 300)]


In [63]:
#insight: any region which includes a border will be infinite
blacklist = set()

for x in range(xmin, xmax+1):
    for location in matrix[x, ymin]: 
        blacklist.add(location)
    for location in matrix[x, ymax]:
        blacklist.add(location)
for y in range(ymin, ymax+1): 
    for location in matrix[xmin, y]: 
        blacklist.add(location)
    for location in matrix[xmax, y]:
        blacklist.add(location)
        
list(blacklist)[0:4]

[(354, 92), (60, 42), (142, 50), (56, 197)]

In [65]:
#count the number of squares to which each location is nearest to, by location
distances = dict()

for x in range(xmin, xmax+1): 
    for y in range(ymin, ymax+1):
        #ignore is several locations have are nearest to this spot
        if len(matrix[x, y]) == 1:
            if matrix[x,y][0] in blacklist: 
                continue
            if matrix[x,y][0] not in distances: 
                distances[matrix[x,y][0]] = 0
            distances[matrix[x,y][0]] += 1
   

In [70]:
for location in distances: 
    if distances[location] == max(distances.values()):
        print(location, distances[location])

(177, 248) 3969


## Part 2


In [72]:
threshold = 10000

In [73]:
matrix = dict()

for x in range(xmin, xmax+1): 
    for y in range(ymin, ymax+1):
        matrix[x,y] = 0
        for location in coordinates: 
            matrix[x,y] += distance(location, (x,y))
            if matrix[x,y] >= threshold:
                break

In [79]:
locations = list()

for x in range(xmin, xmax+1): 
    for y in range(ymin, ymax+1):
        if matrix[x,y] >= threshold: 
            continue
        locations.append((x, y))

len(locations), locations[0:4]

(42123, [(58, 153), (58, 154), (58, 155), (58, 156)])

In [98]:
#check that two regions (list of locations) are contiguous
def iscontiguous(this, that):
    for loc1 in this: 
        for loc2 in that: 
            if distance(loc1, loc2) <= 1: 
                return True
    return False

In [127]:
regions = [[location] for location in locations]

class BreakException(Exception):
    pass


def reduce(regions):
    while True:
        length = len(regions)
        print(str(length).ljust(10), end="\r")
        try: 
            for i, region in enumerate(regions): 
                for j, other in enumerate(regions):
                    if i == j: 
                        continue
                    if iscontiguous(region, other):
                        #prepend other to boost algorithmic efficiency
                        regions[i] =  other + region
                        del regions[j]
                        raise BreakException
        except BreakException: 
            pass
        if len(regions) == length: 
            return regions

reduced = reduce(regions)

In [130]:
lengths = [len(region) for region in reduced]
max(lengths)

42123